Установка зависимостей

In [ ]:
!pip3.9 install -r ./requirements.txt --extra-index-url https://download.pytorch.org/whl/cu116

Прослушивание новых после генерации голосов

In [ ]:
from ss14tts import model, speakers
from IPython.display import Audio, display
from src.SpeakerPatch import dynVoices
import random, glob, os

sample_rate = 24000
example_text = ["Съешь же ещё этих мягких французских булок, да выпей чаю.",
        "Клоун, прекрати разбрасывать банановые кожурки офицерам под ноги!",
        "Капитан, вы уверены что хотите назначить клоуна на должность главы персонала?",
        "Эс Бэ! Тут человек в сером костюме, с тулбоксом и в маске! Помогите!!"]

for voiceFile in glob.glob(dynVoices+'/tmp/*.pt'):
    audio = model.apply_tts(ssml_text='<speak><prosody rate="fast">'+random.choice(example_text)+'</prosody></speak>', speaker="random",voice_path=voiceFile,sample_rate=sample_rate)
    speaker = os.path.relpath(voiceFile,dynVoices)
    print('speaker {}'.format(speaker))
    display(Audio(audio, rate=sample_rate))


Генератор голосов

In [ ]:
from ss14tts import model
from src.AutoGenerator import Generate

for voiceFile in Generate(model,20):  # 20 голосов создает, голоса будет идти с 1
    print(voiceFile)


## Тестирование голосов

In [ ]:
from ss14tts import model, speakers
from IPython.display import Audio, display
from src.SpeakerPatch import dynVoices
import random, glob, os

sample_rate = 24000
example_text = ["Съешь же ещё этих мягких французских булок, да выпей чаю.",
        "Клоун, прекрати разбрасывать банановые кожурки офицерам под ноги!",
        "Капитан, вы уверены что хотите назначить клоуна на должность главы персонала?",
        "Эс Бэ! Тут человек в сером костюме, с тулбоксом и в маске! Помогите!!"]

for voiceFile in glob.glob(dynVoices+'/*.pt'):
    speaker = os.path.relpath(voiceFile,dynVoices)
    print('speaker {} with {}'.format(speaker, voiceFile))
    for example in example_text:
        audio = model.apply_tts(ssml_text='<speak><prosody rate="fast">'+example+'</prosody></speak>', speaker="random",voice_path=voiceFile,sample_rate=sample_rate)
        display(Audio(audio, rate=sample_rate))


## Тест эффекта анонса

In [ ]:
from ss14tts import model, speakers
from IPython.display import Audio, display
from src.SoundEffects import add_echo
import io
import torchaudio
import librosa

sample_rate = 24000
example_text = ["ярких вспышках света",
        "ярких вспышках света",
        "странных звуках, доносящихся из технических туннелей",
        "неестественных конструкциях",
        "непостижимых инопланетных объектах",
        "незнакомых предметах в странных местах",
        "неизвестных инопланетных артефактах",
        "взрывах ламп, сопровождаемых странными звуками"
        ]

for example in example_text:
    audio = model.apply_tts(ssml_text='<speak><prosody rate="fast">На станции обнаружена нестабильная блюспейс аномалия. Пожалуйста, незамедлительно сообщайте научному отделу о '+example+'</prosody></speak>', speaker="baya",sample_rate=sample_rate)
    # Saving to bytes buffer
    buffer_ = io.BytesIO()
    torchaudio.save(buffer_, audio.unsqueeze(0), sample_rate, format="ogg")
    buffer_.seek(0)
    # Параметры эффекта
    new_pitch = 0.8
    decay = 0.3
    buffer_ = add_echo(buffer_, new_pitch, decay)
    display(Audio(buffer_.getvalue(), rate=sample_rate))
